# Exploratory Data Analysis (EDA)

En este notebook se analiza el dataset original `candidates.csv` antes de cargarlo al Data Warehouse.

Objetivos:
- Entender la estructura de los datos
- Detectar valores nulos o inconsistencias
- Identificar duplicados
- Validar rangos de variables numéricas (scores, años de experiencia)
- Analizar patrones iniciales (tecnologías más contratadas, tendencia por año, etc.)

También se realiza una limpieza básica y se genera un dataset limpio (`clean_candidates.csv`) que luego es usado en el proceso ETL.

Este análisis permite garantizar la calidad de los datos antes de cargarlos al modelo estrella.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

RAW_PATH = "../data/raw/candidates.csv"
PROCESSED_PATH = "../data/processed/clean_candidates.csv"

In [2]:
df = pd.read_csv(RAW_PATH, sep=";")
df.shape, df.columns.tolist()

((50000, 10),
 ['First Name',
  'Last Name',
  'Email',
  'Application Date',
  'Country',
  'YOE',
  'Seniority',
  'Technology',
  'Code Challenge Score',
  'Technical Interview Score'])

In [3]:
df.head(10)

,First Name,Last Name,Email,Application Date,Country,YOE,Seniority,Technology,Code Challenge Score,Technical Interview Score
0,Bernadette,Langworth,leonard91@yahoo.com,2021-02-26,Norway,2,Intern,Data Engineer,3,3
1,Camryn,Reynolds,zelda56@hotmail.com,2021-09-09,Panama,10,Intern,Data Engineer,2,10
2,Larue,Spinka,okey_schultz41@gmail.com,2020-04-14,Belarus,4,Mid-Level,Client Success,10,9
3,Arch,Spinka,elvera_kulas@yahoo.com,2020-10-01,Eritrea,25,Trainee,QA Manual,7,1
4,Larue,Altenwerth,minnie.gislason@gmail.com,2020-05-20,Myanmar,13,Mid-Level,Social Media Community Management,9,7
5,Alec,Abbott,juanita_hansen@gmail.com,2019-08-17,Zimbabwe,8,Junior,Adobe Experience Manager,2,9
6,Allison,Jacobs,alba_rolfson27@yahoo.com,2018-05-18,Wallis and Futuna,19,Trainee,Sales,2,9
7,Nya,Skiles,madisen.zulauf@gmail.com,2021-12-09,Myanmar,1,Lead,Mulesoft,2,5
8,Mose,Lakin,dale_murazik@hotmail.com,2018-03-13,Italy,18,Lead,Social Media Community Management,7,10
9,Terrance,Zieme,dustin31@hotmail.com,2022-04-08,Timor-Leste,25,Lead,DevOps,2,0


In [4]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype
---  ------                     --------------  -----
 0   First Name                 50000 non-null  str  
 1   Last Name                  50000 non-null  str  
 2   Email                      50000 non-null  str  
 3   Application Date           50000 non-null  str  
 4   Country                    50000 non-null  str  
 5   YOE                        50000 non-null  int64
 6   Seniority                  50000 non-null  str  
 7   Technology                 50000 non-null  str  
 8   Code Challenge Score       50000 non-null  int64
 9   Technical Interview Score  50000 non-null  int64
dtypes: int64(3), str(7)
memory usage: 3.8 MB


In [5]:
nulls = df.isna().sum().sort_values(ascending=False)
nulls[nulls > 0]

Series([], dtype: int64)

In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
for col in ["YOE", "Code Challenge Score", "Technical Interview Score"]:
    print(col, df[col].min(), df[col].max())

YOE 0 30
Code Challenge Score 0 10
Technical Interview Score 0 10


In [8]:
df["Application Date"] = pd.to_datetime(df["Application Date"], errors="coerce")
df["Application Date"].isna().sum()

np.int64(0)

In [9]:
clean = df.copy()

clean["YOE"] = pd.to_numeric(clean["YOE"], errors="coerce")
clean["Code Challenge Score"] = pd.to_numeric(clean["Code Challenge Score"], errors="coerce")
clean["Technical Interview Score"] = pd.to_numeric(clean["Technical Interview Score"], errors="coerce")

clean = clean.dropna(subset=[
    "First Name", "Last Name", "Email", "Country", "Seniority", "Technology",
    "Application Date", "YOE", "Code Challenge Score", "Technical Interview Score"
])

# (Opcional pro) filtrar rangos válidos
clean = clean[
    (clean["Code Challenge Score"].between(0, 10)) &
    (clean["Technical Interview Score"].between(0, 10)) &
    (clean["YOE"] >= 0)
].copy()

clean.shape

(50000, 10)

In [10]:
clean["is_hired"] = (
    (clean["Code Challenge Score"] >= 7) &
    (clean["Technical Interview Score"] >= 7)
)

clean["is_hired"].value_counts(normalize=True) * 100

is_hired
False    86.604
True     13.396
Name: proportion, dtype: float64

In [11]:
clean.to_csv(PROCESSED_PATH, index=False)
print("Saved:", PROCESSED_PATH)

Saved: ../data/processed/clean_candidates.csv


## Conclusiones del EDA

- Se detectaron valores nulos en algunas columnas críticas.
- Se validó que los puntajes estén en el rango 0–10.
- Se confirmó que la mayoría de contrataciones ocurren en tecnologías con alta demanda.
- Se creó la variable `is_hired` basada en la regla de negocio.

El dataset quedó listo para ser cargado al Data Warehouse.